Scraping data from Wikipedia list.

In [20]:
import pandas as pd

 
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
tables = pd.read_html(url)[0]
tables.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Removing postcodes with unassigned boroughs and combining rows with a shared postcode.

In [24]:
tables = tables[tables['Borough'] != "Not assigned"]
tables = tables.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()


In [25]:
tables.shape

(103, 3)